In [58]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split

In [59]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Assuming the first column is the label
train_labels = train_data.iloc[:, 0].values
train_features = train_data.iloc[:, 1:].values.reshape(-1, 28, 28).astype('float32')

# Split train data for training and validation
X_train, X_val, y_train, y_val = train_test_split(train_features, train_labels, test_size=0.2)

Data set for Data loader

In [60]:
class CustomDataset:
    def __init__(self, features, labels=None, transform=None):
        self.features = features
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        image = self.features[idx].reshape(28, 28, 1)  # Reshape to 28x28
        if self.transform:
            image = self.transform(image)

        if self.labels is not None:
            label = self.labels[idx]
            return image, label
        else:
            return image

In [61]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = CustomDataset(X_train, y_train, transform=transform)
val_dataset = CustomDataset(X_val, y_val, transform=transform)
test_dataset = CustomDataset(test_data.values.reshape(-1, 28, 28).astype('float32'), transform=transform)

# Data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=1000, shuffle=False,drop_last=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=1000, shuffle=False)


Defining Model


In [62]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(1024, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

initializing model, optimizer, loss

In [63]:
model = Net()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [64]:
one_batch = next(iter(val_loader))

# Check the output
print(type(one_batch))  # Should be a tuple
print(one_batch[0].shape)

<class 'list'>
torch.Size([1000, 1, 28, 28])


In [65]:
def train(epoch):

    for batch_idx,batch  in enumerate(train_loader):

        data = batch[0]
        labels = batch[1]
        optimizer.zero_grad()
        
        out = model(data)

        loss = criterion(out,labels)

        loss.backward()

        optimizer.step()

        if batch_idx % 100 == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}")
 


In [66]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in val_loader:
            
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(val_loader.dataset)
    

    
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(val_loader.dataset)} ({100. * correct / len(val_loader.dataset):.0f}%)\n')

# Training and testing the model


In [67]:
for epoch in range(1, 10):
    train(epoch)
    

Train Epoch: 1 [0/33600 (0%)]	Loss: 45.958023
Train Epoch: 1 [6400/33600 (19%)]	Loss: 0.171683
Train Epoch: 1 [12800/33600 (38%)]	Loss: 0.093988
Train Epoch: 1 [19200/33600 (57%)]	Loss: 0.207935
Train Epoch: 1 [25600/33600 (76%)]	Loss: 0.078447
Train Epoch: 1 [32000/33600 (95%)]	Loss: 0.124650
Train Epoch: 2 [0/33600 (0%)]	Loss: 0.174908
Train Epoch: 2 [6400/33600 (19%)]	Loss: 0.166218
Train Epoch: 2 [12800/33600 (38%)]	Loss: 0.043634
Train Epoch: 2 [19200/33600 (57%)]	Loss: 0.115782
Train Epoch: 2 [25600/33600 (76%)]	Loss: 0.054686
Train Epoch: 2 [32000/33600 (95%)]	Loss: 0.233528
Train Epoch: 3 [0/33600 (0%)]	Loss: 0.046297
Train Epoch: 3 [6400/33600 (19%)]	Loss: 0.035829
Train Epoch: 3 [12800/33600 (38%)]	Loss: 0.028259
Train Epoch: 3 [19200/33600 (57%)]	Loss: 0.140524
Train Epoch: 3 [25600/33600 (76%)]	Loss: 0.068254
Train Epoch: 3 [32000/33600 (95%)]	Loss: 0.097789
Train Epoch: 4 [0/33600 (0%)]	Loss: 0.058639
Train Epoch: 4 [6400/33600 (19%)]	Loss: 0.004078
Train Epoch: 4 [12800/3

In [70]:
test()


Test set: Average loss: 0.0001, Accuracy: 7851/8400 (93%)



In [72]:
def predict():
    model.eval()
    predictions = []
    with torch.no_grad():
        for data in test_loader:
            
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            predictions.extend(pred.tolist())

    return predictions

In [73]:
test_predictions = predict()

In [74]:
predictions_df = pd.DataFrame({
    'ImageId': range(1, len(test_predictions) + 1),  # ImageId starts at 1, not 0
    'Label': [pred[0] for pred in test_predictions]  # Assuming test_predictions is a list of lists
})

# Write the DataFrame to a CSV file
predictions_df.to_csv('submission.csv', index=False)